In [81]:
import pandas as pd
import numpy as np
import yfinance as yf
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import Callback, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Bidirectional, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import Huber
from tensorflow.keras.regularizers import l2
import tensorflow as tf



In [82]:
start_date = "2022-01-03"
end_date   = "2024-11-01"

data = yf.download("AAPL", start=start_date, end=end_date)
data.columns = data.columns.get_level_values(0) # from multi index to single index
print(data.head())
data_copy = data.copy()

[*********************100%***********************]  1 of 1 completed

Price                       Adj Close       Close        High         Low  \
Date                                                                        
2022-01-03 00:00:00+00:00  179.273621  182.009995  182.880005  177.710007   
2022-01-04 00:00:00+00:00  176.998337  179.699997  182.940002  179.119995   
2022-01-05 00:00:00+00:00  172.290192  174.919998  180.169998  174.639999   
2022-01-06 00:00:00+00:00  169.414093  172.000000  175.300003  171.639999   
2022-01-07 00:00:00+00:00  169.581558  172.169998  174.139999  171.029999   

Price                            Open     Volume  
Date                                              
2022-01-03 00:00:00+00:00  177.830002  104487900  
2022-01-04 00:00:00+00:00  182.630005   99310400  
2022-01-05 00:00:00+00:00  179.610001   94537600  
2022-01-06 00:00:00+00:00  172.699997   96904000  
2022-01-07 00:00:00+00:00  172.889999   86709100  


In [83]:
# data['days_range'] = data['High'] - data['Low']   # calculating the range of the day
# data['yesterdays_close'] = data['Adj Close'].shift(1)
# data['jump_from_yesterday'] = data['Open']- data['yesterdays_close']
# data['days_movement'] = data['Adj Close']-data['Open']
# data = data.drop(columns=['yesterdays_close','Close','High','Low','Open'])

# data_v1 = data.copy()
# del(data)

In [84]:
data = data_copy.copy()

data['target'] = data['Adj Close'].shift(-1)
data = data[data['target'].notnull()]
data = data.drop(columns=['Close','High','Low','Open','Volume'])
data = data.reset_index()
data['Date'] = data['Date'].dt.strftime('%Y-%m-%d')
data['Date'] = pd.to_datetime(data['Date']).dt.date
data.set_index('Date', inplace=True)



In [85]:
data.head()

Price,Adj Close,target
Date,,
2022-01-03,179.273621,176.998337
2022-01-04,176.998337,172.290192
2022-01-05,172.290192,169.414093
2022-01-06,169.414093,169.581558
2022-01-07,169.581558,169.601257


In [86]:

# Assume start_date and end_date are defined, and data is your DataFrame
a = pd.date_range(start=start_date, end=end_date, freq="D")  # continuous dates
b = data.index  # our time series
diff_dates = a.difference(b)  # finds what in 'a' is not in 'b'

# Ensure diff_dates remains as Timestamps for compatibility with DataFrame index
# (no need to convert to string and back to datetime)
diff_dates = pd.to_datetime(diff_dates).date

# Ensure `td` is defined correctly
td = pd.Timedelta(days=1)  # Adjust according to your needs

for date in diff_dates:
    prev_date = date - td  # Previous date
    # Check if the previous date exists in the index
    if prev_date in data.index:  # prev_date is still a Timestamp
        prev_val = data.loc[prev_date]  # Access using loc
        data.loc[date] = prev_val  # Impute previous value
    else:
        print(f"Previous date {prev_date} not found in index.")  # Debug message or handling

data.sort_index(inplace=True)  # Sort the index
data.freq = "D"  # Set the time index frequency as daily


In [87]:
# Define split date and convert to date-only format to match the index
val_split_date = pd.to_datetime('2023-12-31').date()
test_split_date = pd.to_datetime('2024-06-30').date()

# Split the data
train = data[:val_split_date]  # Data up to and including 2023-12-31
val = data[val_split_date:test_split_date]   # Data from 2023-12-31 onward
test = data[test_split_date:]   # Data from 2023-12-31 onward




In [88]:

scaler_train = MinMaxScaler()
values = scaler_train.fit_transform(train[['Adj Close']])


# Define the window size
WINDOW = 14  # Window size of 14 days


train_data = tf.data.Dataset.from_tensor_slices(values) # Create a TensorFlow Dataset from the array
train_data = train_data.window(WINDOW + 1, shift=1, drop_remainder=True) # Create windowed dataset with the specified window size
train_data = train_data.flat_map(lambda x: x.batch(WINDOW + 1)) # Flatten the windowed dataset by batching

# Create features and target tuple
train_data = train_data.map(lambda x: (x[:-1], x[-1]))  # x[:-1] for features, x[-1, 1] for target 'Adj Close' # Here, we use all columns for features, but only the 'Adj Close' (index 1) as the target
train_data = train_data.batch(32).prefetch(1) # Create batches of windows



scaler_val = MinMaxScaler()
val_values = scaler_val.fit_transform(val[[ 'Adj Close']])

# Convert to TensorFlow Datasets similarly as before
#val_values = val[['Volume', 'Adj Close']].values
val_data = tf.data.Dataset.from_tensor_slices(val_values)
val_data = val_data.window(WINDOW + 1, shift=1, drop_remainder=True).flat_map(lambda x: x.batch(WINDOW + 1))
val_data = val_data.map(lambda x: (x[:-1], x[-1])).batch(32).prefetch(1)


In [89]:

# Custom callback
class CustomCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs.get('mae') < 0.1:
            print("MAE under 0.1... Stopping training")
            self.model.stop_training = True

my_callback = CustomCallback()

# Learning rate scheduler
def scheduler(epoch, lr):
    if epoch < 2:
        return 0.01
    else:
        return lr * 0.99

lr_scheduler = LearningRateScheduler(scheduler)

# LSTM model definition
lstm_model = Sequential([
    Bidirectional(LSTM(128, return_sequences=True), input_shape=[WINDOW,1]),
    Dropout(0.1),
    BatchNormalization(),
    Bidirectional(LSTM(128, return_sequences=True), input_shape=[WINDOW,1]),
    Dropout(0.1),
    BatchNormalization(),
    Bidirectional(LSTM(128)),
    Dropout(0.1),
    BatchNormalization(),
    Dense(256, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.1),  # Experiment with different dropout rates
    Dense(1)
])

# Compile model
lstm_model.compile(
    loss=Huber(),
    optimizer=Adam(),
    metrics=['mae']
)

# Callbacks for early stopping and learning rate reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# Model summary
lstm_model.summary()


/opt/anaconda3/envs/Conda_3_12_7/lib/python3.12/site-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional_16                │ (None, 14, 256)        │       133,120 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_23 (Dropout)            │ (None, 14, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 14, 256)        │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_17                │ (None, 14, 256)        │       394,240 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_24 (Dropout)            │ (None, 14, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 14, 256)        │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_18                │ (None, 256)            │       394,240 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_25 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_14          │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_26 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 990,721 (3.78 MB)

 Trainable params: 989,185 (3.77 MB)

 Non-trainable params: 1,536 (6.00 KB)

In [90]:
# lstm_history = lstm_model.fit(
#     train_data,
#     epochs=100,
#     callbacks=[lr_scheduler, my_callback]
# )

# Fit the model with training data
lstm_history = lstm_model.fit(
    train_data,
    epochs=100,
    validation_data=val_data,  # Ensure you define val_data appropriately
    callbacks=[lr_scheduler, my_callback, early_stopping, reduce_lr],
    batch_size=32  # Optional: specify batch size if it's not handled in your Dataset
)


Epoch 1/100
     21/Unknown 3s 26ms/step - loss: 3.8053 - mae: 2.0083

/opt/anaconda3/envs/Conda_3_12_7/lib/python3.12/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - loss: 3.6673 - mae: 1.9060 - val_loss: 1.3525 - val_mae: 0.5881 - learning_rate: 0.0100
Epoch 2/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.9403 - mae: 0.2613 - val_loss: 0.5434 - val_mae: 0.5880 - learning_rate: 0.0100
Epoch 3/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 0.3292 - mae: 0.3378 - val_loss: 0.1182 - val_mae: 0.2054 - learning_rate: 0.0099
Epoch 4/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 0.1093 - mae: 0.2310 - val_loss: 0.0694 - val_mae: 0.2149 - learning_rate: 0.0098
Epoch 5/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - loss: 0.0623 - mae: 0.2069 - val_loss: 0.0451 - val_mae: 0.2040 - learning_rate: 0.0097
Epoch 6/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - loss: 0.0372 - mae: 0.1838 - val_loss: 0.0426 - val_mae: 0.2177 - learning_rate: 0.0096
Epoch 7/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - loss: 0.0316 - mae: 0.1826 - val_loss: 0.0407 - val_mae: 0.2215 - learning_rate: 0.0095
Epoch 8/100
23/23 

In [91]:
# Assuming lstm_model is your trained model
model_save_path = '/Users/monilshah/Documents/02_NWU/09_MSDS_458_DL/99_group_project/StockPricePrediction/01_Bidirectional_LSTM/01_Models/bidirecttional_lstm_1d.keras'  # Specify your desired path here

# Save the model
lstm_model.save(model_save_path)

print(f'Model saved to {model_save_path}')


Model saved to /Users/monilshah/Documents/02_NWU/09_MSDS_458_DL/99_group_project/StockPricePrediction/01_Bidirectional_LSTM/01_Models/bidirecttional_lstm_1d.keras


## Test

In [92]:
# Assuming 'test' is your test DataFrame and scaler_train is your fitted MinMaxScaler

# Scale the test data
test_values = scaler_val.transform(test[['Adj Close']])  # Use only the features for scaling

# Define the window size
WINDOW = 30  # Same window size as used for training"

# Create a TensorFlow Dataset from the scaled test data
test_data = tf.data.Dataset.from_tensor_slices(test_values)

# Create windowed dataset with the specified window size
test_data = test_data.window(WINDOW + 1, shift=1, drop_remainder=True)

# Flatten the windowed dataset by batching
test_data = test_data.flat_map(lambda x: x.batch(WINDOW + 1))

# Create features and target tuple (for evaluation, you can use the last value as target)
test_data = test_data.map(lambda x: (x[:-1], x[-1]))  # Here, x[-1, 1] corresponds to the 'Adj Close' price

# Create batches of windows
test_data = test_data.batch(32).prefetch(1)  # Adjust batch size as necessary


In [93]:
predictions = lstm_model.predict(test_data)


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 134ms/step


/opt/anaconda3/envs/Conda_3_12_7/lib/python3.12/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


In [94]:
predictions

array([[0.47606865],
       [0.47610283],
       [0.47612596],
       [0.47613373],
       [0.47614127],
       [0.47618377],
       [0.47618034],
       [0.47619095],
       [0.4762174 ],
       [0.47624117],
       [0.4762747 ],
       [0.47622025],
       [0.47624543],
       [0.4762488 ],
       [0.47626638],
       [0.47631395],
       [0.47630706],
       [0.47623634],
       [0.4761938 ],
       [0.47619587],
       [0.47620165],
       [0.4762061 ],
       [0.47620672],
       [0.47620228],
       [0.47614694],
       [0.47614056],
       [0.47614294],
       [0.47614062],
       [0.47614446],
       [0.476151  ],
       [0.47616228],
       [0.47618335],
       [0.47616386],
       [0.4761776 ],
       [0.47617358],
       [0.47614685],
       [0.476076  ],
       [0.47606707],
       [0.47607803],
       [0.47609466],
       [0.47610557],
       [0.4761    ],
       [0.47609702],
       [0.47611073],
       [0.47613955],
       [0.47614872],
       [0.47617853],
       [0.476

In [95]:
# Reshape predictions to match the expected input shape of the scaler
predictions_reshaped = np.zeros((predictions.shape[0], 2))  # Create an array for two features
predictions_reshaped[:, 0] = predictions.flatten()  # Place the predictions in the first column

# Inverse scale the predictions
predictions_original = scaler_val.inverse_transform(predictions_reshaped)

# Extract the first column for actual sales predictions
predictions_original = predictions_original[:, 0]

# Output the original predictions
print(predictions_original)

[189.26230082 189.26407266 189.26527139 189.26567458 189.2660654
 189.26826823 189.26809058 189.26864051 189.27001071 189.27124343
 189.27298128 189.27015901 189.27146433 189.27163889 189.2725503
 189.27501573 189.27465889 189.27099318 189.26878726 189.2688954
 189.26919508 189.26942525 189.26945769 189.26922752 189.2663589
 189.26602833 189.26615191 189.26603142 189.26623069 189.26656899
 189.26715445 189.2682466  189.26723633 189.26794846 189.26773992
 189.26635427 189.26268238 189.26221895 189.26278742 189.2636494
 189.26421478 189.26392591 189.26377143 189.26448202 189.2659758
 189.26645159 189.26799635 189.26861889 189.26862352 189.26876255
 189.26885832 189.26914719 189.26873165 189.26774301 189.26861734
 189.2686158  189.26869767 189.26915955 189.26967241 189.26945923
 189.27122798 189.27111367 189.27141644 189.27155084 189.27068423
 189.2674078  189.26674818 189.26733365 189.26665241 189.26649793
 189.26640988 189.26647013 189.26639289 189.26732129 189.26724868
 189.26707876 18

In [96]:
y_test = test['Adj Close']
y_test

Date
2024-06-30    210.376480
2024-07-01    216.499405
2024-07-02    220.015335
2024-07-03    221.293854
2024-07-04    221.293854
                 ...    
2024-10-28    233.399994
2024-10-29    233.669998
2024-10-30    230.100006
2024-10-31    230.100006
2024-11-01    230.100006
Name: Adj Close, Length: 125, dtype: float64